In [15]:
# Bibliotecas utilizadas

from dash import Dash, dcc, html, Input, Output, callback, State
import os
import pandas as pd
import plotly.express as px
from PIL import Image
import glob as gb

In [6]:
# Código armazenar as imagens de capa do site

pil_image_dia = Image.open("assets/Capa-pcd-dia.png")

pil_image_noite = Image.open("assets/Capa-pcd-noite.png")

pil_image_tarde = Image.open("assets/Capa-pcd-tarde.png")

In [7]:
#Função que define as atividades de acordo com o clima (temperatura e chuva)

def obter_atividades_ludicas(temperatura, chuva):
    """Retorna uma lista de atividades lúdicas recomendadas com base na temperatura e precipitação."""

    atividades = []

    if temperatura > 25:  # Dias Quentes
        if not chuva:
            atividades.extend([
                "Piquenique no parque", "Jogos de bola", "Andar de bicicleta", "Nadar", "Slackline", "Frescobol",
                "Cozinhar receitas refrescantes", "Fazer experiências com água e gelo"
            ])
        else:
            atividades.extend([
                "Sessão de cinema com pipoca", "Maratona de jogos de tabuleiro", "Criar um forte de cobertores",
                "Festa do pijama", "Aprender origami", "Karaokê"
            ])
            if chuva == "leve":
                atividades.append("Caminhada na chuva com galochas")

    elif 15 <= temperatura <= 25:  # Dias Agradáveis
        if not chuva:
            atividades.extend([
                "Trilha leve", "Passeio de bicicleta", "Explorar um novo bairro", "Visitar um jardim botânico",
                "Cozinhar uma receita nova", "Fazer um quebra-cabeça", "Aprender um novo idioma"
            ])
        else:
            atividades.extend([
                "Ler um livro com uma bebida quente", "Noite de jogos com amigos", "Aprender um instrumento",
                "Fazer artesanato", "Assistir a um documentário"
            ])
            if chuva == "leve":
                atividades.append("Fotografar a natureza")

    else:  # Dias Frios
        if not chuva:
            atividades.extend([
                "Patinação no gelo", "Esquiar (se houver neve)", "Fazer um boneco de neve", "Visitar um mercado de Natal",
                "Assar biscoitos", "Fazer fondue", "Construir um iglu de papelão"
            ])
        else:
            atividades.extend([
                "Maratona de filmes ou séries", "Ler um livro enrolado em um cobertor", "Aprender a tricotar",
                "Spa day em casa", "Organizar o armário", "Escrever um diário"
            ])

    return atividades

texto_atividades = 'Aqui estarão as atividades recomendadas para o dia selecionado.'


In [23]:
# Dicionário para mapear siglas para nomes de capitais formatados
substituicoes = {'BRASILIA': 'Brasília','GOIANIA': 'Goiânia','CAMPO GRANDE': 'Campo Grande','CUIABA': 'Cuiabá','MACEIO': 'Maceió','SALVADOR': 'Salvador','FORTALEZA': 'Fortaleza','SAO LUIS': 'São Luís','JOAO PESSOA': 'João Pessoa','PALMARES': 'Palmares','TERESINA': 'Teresina','NATAL': 'Natal','ARACAJU': 'Aracaju','RIO BRANCO': 'Rio Branco','MANAUS': 'Manaus','MACAPA': 'Macapá',
    'BELEM': 'Belém','PORTO VELHO': 'Porto Velho','BOA VISTA': 'Boa Vista','PALMAS': 'Palmas','VITORIA': 'Vitória','BELO HORIZONTE': 'Belo Horizonte','RIO DE JANEIRO': 'Rio de Janeiro','SAO PAULO': 'São Paulo','CURITIBA': 'Curitiba','PORTO ALEGRE': 'Porto Alegre','FLORIANOPOLIS': 'Florianópolis'
}

# Cria um dicionário vazio para armazenar as capitais e seus respectivos arquivos
capitais_por_arquivo = {}

# Lista de nomes de arquivos
arquivos = gb.glob("*.csv")

# Itera sobre a lista de nomes de arquivos
for arquivo in arquivos:
    partes_nome_arquivo = arquivo.split('_')  # Divide o nome do arquivo
    capital_sem_acento = partes_nome_arquivo[4].upper()  # Extrai o nome da capital

    # Corrige o nome da capital usando o dicionário de substituições
    capital = substituicoes.get(capital_sem_acento)

    capitais_por_arquivo[capital] = arquivo  # Adiciona ao dicionário

print(capitais_por_arquivo)

{'Brasília': 'INMET_CO_DF_A001_BRASILIA_01-01-2024_A_30-04-2024.CSV', 'Goiânia': 'INMET_CO_GO_A002_GOIANIA_01-01-2024_A_30-04-2024.CSV', 'Campo Grande': 'INMET_CO_MS_A702_CAMPO GRANDE_01-01-2024_A_30-04-2024.CSV', 'Cuiabá': 'INMET_CO_MT_A901_CUIABA_01-01-2024_A_30-04-2024.CSV', 'Maceió': 'INMET_NE_AL_A303_MACEIO_01-01-2024_A_30-04-2024.CSV', 'Salvador': 'INMET_NE_BA_A401_SALVADOR_01-01-2024_A_30-04-2024.CSV', 'Fortaleza': 'INMET_NE_CE_A305_FORTALEZA_01-01-2024_A_30-04-2024.CSV', 'São Luís': 'INMET_NE_MA_A203_SAO LUIS_01-01-2024_A_30-04-2024.CSV', 'João Pessoa': 'INMET_NE_PB_A320_JOAO PESSOA_01-01-2024_A_30-04-2024.CSV', 'Palmares': 'INMET_NE_PE_A357_PALMARES_01-01-2024_A_30-04-2024.CSV', 'Teresina': 'INMET_NE_PI_A312_TERESINA_01-01-2024_A_30-04-2024.CSV', 'Natal': 'INMET_NE_RN_A304_NATAL_01-01-2024_A_30-04-2024.CSV', 'Aracaju': 'INMET_NE_SE_A409_ARACAJU_01-01-2024_A_30-04-2024.CSV', 'Rio Branco': 'INMET_N_AC_A104_RIO BRANCO_01-01-2024_A_30-04-2024.CSV', 'Manaus': 'INMET_N_AM_A101_MANAU

In [9]:
# Ler o arquivo para ver o formato dos dados
df = pd.read_csv('INMET_CO_DF_A001_BRASILIA_01-01-2024_A_30-04-2024.CSV', skiprows=8, delimiter=';', encoding='latin1')

# Abandonar uma coluna sem nome
df.drop(columns=['Unnamed: 19'], inplace=True)

# Converter 'data' e 'hora' pro formato de data e hora
df['Data'] = pd.to_datetime(df['Data'], format='%Y/%m/%d')
df['Hora UTC'] = pd.to_datetime(df['Hora UTC'], format='%H%M UTC')

# Extrair hora de 'Hora UTC' e criar uma nova coluna chamada 'Hora'
df['Hora'] = df['Hora UTC'].dt.hour

# criar coluna 'Periodo' com base na hora
def categorize_hour(hour):
    if 0 <= hour <= 5:
        return 'Madrugada'
    elif 6 <= hour <= 11:
        return 'Manhã'
    elif 12 <= hour <= 17:
        return 'Tarde'
    else:
        return 'Noite'

df['Periodo'] = df['Hora'].apply(categorize_hour)

# Agrupar por 'Data' e 'Periodo' e calcular a média de cada coluna numérica
df_grouped = df.groupby(['Data', 'Periodo']).agg(
    Precipitacao=('PRECIPITAÇÃO TOTAL. HORÁRIO (mm)', 'mean'),
    Radiacao=('RADIACAO GLOBAL (Kj/m²)', 'mean'),
    Temp_Max=('TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)', 'mean'),
    Temp_Min=('TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)', 'mean'),
    Umidade=('UMIDADE RELATIVA DO AR. HORARIA (%)', 'mean'),
    Vento_Vel=('VENTO. VELOCIDADE HORARIA (m/s)', 'mean'),
    Vento_Raj=('VENTO. RAJADA MAXIMA (m/s)', 'mean')
).reset_index().round(2)

# Criar um dicionário para mapear os números dos meses para os nomes dos meses em português
meses_pt = {
    1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril',
    5: 'Maio', 6: 'Junho', 7: 'Julho', 8: 'Agosto',
    9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
}

# Extract month from the `Data` column and create a new column called 'Mês' at the beginning
df_grouped['Mês'] = df_grouped['Data'].dt.month.map(meses_pt)
df_grouped.insert(0, 'Mês', df_grouped.pop('Mês'))

df_grouped['Dia'] = df_grouped['Data'].dt.strftime('%d-%a')

df_grouped['Data'] = df_grouped['Data'].dt.day.astype(str)



In [10]:
# Períodos para filtragem
todos_os_periodos = df_grouped['Periodo'].unique()
for i in todos_os_periodos:
    print(i)
    
todos_os_periodos = ['Todos os períodos'] + list(todos_os_periodos)

# Dias para filtragem
todos_os_dias = []
for i in range(0,30):
    numero = str(i+1)
    todos_os_dias.append(numero)
todos_os_dias = ['Todos os dias'] + todos_os_dias

# Meses para filtragem
meses = list(meses_pt.values())


Madrugada
Manhã
Noite
Tarde


In [11]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8060/ in your web browser.

app = Dash(__name__)

colors = {
    'background': '#99EEF1',
    'text': '#0C3132'
}

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options

figt = px.line(df_grouped, x="Data", y="Temp_Max", color="Periodo", symbol = 'Periodo',title = 'Temperatura Máxima por Período (ºC)')

figt.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

figchu = px.line(df_grouped, x="Data", y="Precipitacao", color="Periodo", symbol = 'Periodo',title = 'Precipitação por Período (mm)')

figchu.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

app.layout = html.Div(id = 'tudo', style={'backgroundColor': colors['background']}, children=[

    dcc.Store(id='cor_da_letra', data=colors['text']),
    dcc.Store(id='cor_do_fundo', data=colors['background']),

    html.H1(
        id = 'titulo',
        children='CLIMA LÚDICO',
        style={
            'textAlign': 'center',
            'color': colors['text'],
            'fontSize': 45
        }
    ),

    html.Img(id = 'imagem-capa', src=pil_image_dia, width= 1100, style = {'display': 'block','margin':'auto'}),

    html.Div(id = 'texto', children='Planeje a brincadeira ideal para o seu dia!', style={
        'textAlign': 'center',
        'color': colors['text'],
        'fontSize': 60,
        'margin-top': '30px'
    }),
    
    dcc.Store(id='mes_guardado', data='Janeiro'),
    
    dcc.Store(id='dia_guardado', data='Todos os dias'),
    
    dcc.Store(id='período_guardado', data = 'Todos os períodos'),

    dcc.Store(id='cidade_guardada', data = 'Brasília'),

    dcc.Dropdown(
        id='cidade',
        options=[{'label': i, 'value': i} for i in capitais_por_arquivo.keys()],  # Opções do dropdown de cidades
        value='Brasília', 
        placeholder="Selecione uma cidade",
        style={
            'width': '250px',
            'borderRadius': '10px',
            'margin': 'auto',
            'margin-top': '15px',
            'backgroundColor': '#54ACAF'
        }
    ),

    dcc.Dropdown(
        id='mês',
        options=[i for i in meses],
        value='Janeiro',
        placeholder="Selecione um mês",
        style={
            'width': '250px',
            'borderRadius': '10px',
            'margin': 'auto',
            'margin-top': '15px',
            'backgroundColor': '#54ACAF',
            }
    ),
    
    dcc.Dropdown(
        id='período',
        options=todos_os_periodos,
        value='Todos os períodos',
        placeholder="Selecione um período",
        style={
            'width': '250px',
            'borderRadius': '10px',
            'margin': 'auto',
            'margin-top': '15px',
            'backgroundColor': '#54ACAF'
            }
    ),

    html.H2(
                id = 'texto_atividades',
                children='Escolha um dia e um período para ver as atividades recomendadas!',
                style={
                    'textAlign': 'center',
                    'color': colors['text'],
                    'fontSize': 15
                }
            ),
    
    dcc.Dropdown(
        id='dia',
        options=todos_os_dias,
        value='Todos os dias',
        placeholder="Selecione um dia",
        style={
            'width': '250px',
            'borderRadius': '10px',
            'margin': 'auto',
            'margin-top': '15px',
            'backgroundColor': '#54ACAF'
            }
    ),

    html.Button('Atualizar os dados', 
                id='desenhar', 
                n_clicks=0,
                style = {
                    'width': '200px',
                    'borderRadius': '10px',
                    'display': 'block',
                    'margin': 'auto',
                    'margin-top': '20px',
                    'margin-bottom': '20px',
                    'backgroundColor': '#54ACAF',
                }),

    html.H3(
            id = 'texto_atividades_escolhidas',
            children=texto_atividades,
            style={
                'textAlign': 'center',
                'color': colors['text'],
                'fontSize': 15
            }
        ),

    

    dcc.Graph(
        id='GráficoT',
        figure=figt,
        style = {'width': '1000px', 'display': 'block','margin':'auto'}
    ),
    
    dcc.Graph(
        id='GráficoChuva',
        figure=figchu,
        style = {'width': '1000px', 'display': 'block','margin':'auto', 'margin-top': '20px',}
    )
])

@callback(
    Output('mes_guardado', 'data'),
    Input('mês', 'value')
)
def update_output(value):
    return value

@callback(
    Output('período_guardado', 'data'),
    Input('período', 'value')
)
def update_output(value):
    return value

@callback(
    Output('cidade_guardada', 'data'),
    Input('cidade', 'value')
)
def update_output(value):
    return value

@callback(
    Output('dia_guardado', 'data'),
    Input('dia', 'value')
)
def update_output(value):
    return value

@callback(
    Output('GráficoT', 'figure'),
    Output('GráficoChuva', 'figure'),
    Output('imagem-capa', 'src'),
    Output('tudo', 'style'),
    Output('titulo', 'style'),
    Output('texto', 'style'),
    Output('texto_atividades', 'style'),
    Output('texto_atividades_escolhidas', 'style'),
    Output('texto_atividades_escolhidas', 'children'),
    Input('desenhar', 'n_clicks'),
    State('mes_guardado', 'data'),
    State('período_guardado', 'data'),
    State('cidade_guardada', 'data'),
    State('dia_guardado', 'data')
)
def update_output(n_clicks, mes_guardado, período_guardado, cidade_guardada, dia_guardado):

    df = pd.read_csv(capitais_por_arquivo[cidade_guardada], skiprows=8, delimiter=';', encoding='latin1')

    df.drop(columns=['Unnamed: 19'], inplace=True)

    df['Data'] = pd.to_datetime(df['Data'], format='%Y/%m/%d')
    df['Hora UTC'] = pd.to_datetime(df['Hora UTC'], format='%H%M UTC')

    df['Hora'] = df['Hora UTC'].dt.hour

    def categorize_hour(hour):
        if 0 <= hour <= 5:
            return 'Madrugada'
        elif 6 <= hour <= 11:
            return 'Manhã'
        elif 12 <= hour <= 17:
            return 'Tarde'
        else:
            return 'Noite'

    df['Periodo'] = df['Hora'].apply(categorize_hour)

    df_grouped = df.groupby(['Data', 'Periodo']).agg(
        Precipitacao=('PRECIPITAÇÃO TOTAL. HORÁRIO (mm)', 'mean'),
        Radiacao=('RADIACAO GLOBAL (Kj/m²)', 'mean'),
        Temp_Max=('TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)', 'mean'),
        Temp_Min=('TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)', 'mean'),
        Umidade=('UMIDADE RELATIVA DO AR. HORARIA (%)', 'mean'),
        Vento_Vel=('VENTO. VELOCIDADE HORARIA (m/s)', 'mean'),
        Vento_Raj=('VENTO. RAJADA MAXIMA (m/s)', 'mean')
    ).reset_index().round(2)

    meses_pt = {
        1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril',
        5: 'Maio', 6: 'Junho', 7: 'Julho', 8: 'Agosto',
        9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
    }

    df_grouped['Mês'] = df_grouped['Data'].dt.month.map(meses_pt)
    df_grouped.insert(0, 'Mês', df_grouped.pop('Mês'))

    df_grouped['Dia'] = df_grouped['Data'].dt.strftime('%d-%a')

    df_grouped['Data'] = df_grouped['Data'].dt.day.astype(str)

    df_filtrado = df_grouped[(df_grouped['Mês'] == mes_guardado) & 
                            (df_grouped['Periodo'] == período_guardado) &
                            (df_grouped['Data'] == dia_guardado)]

    temperatura = df_filtrado['Temp_Max'].mean()
    chuva = df_filtrado['Precipitacao'].mean() > 0

    if dia_guardado == 'Todos os dias':
        texto_atividades = 'Aqui estarão suas atividades! Selecione um dia e período!'
    
    if dia_guardado != 'Todos os dias':
        atividades = obter_atividades_ludicas(temperatura, chuva)
        texto_atividades = ", ".join(atividades)  


    if período_guardado == 'Todos os períodos':
        df_essemes = df_grouped[(df_grouped["Mês"] == mes_guardado)]
        figt = px.line(df_essemes, x="Data", y="Temp_Max", color="Periodo", symbol = 'Periodo',title = 'Temperatura Máxima (ºC) por Período em ' + mes_guardado + ' na cidade de ' + cidade_guardada)
        figchu = px.line(df_essemes, x="Data", y="Precipitacao", color="Periodo", symbol = 'Periodo',title = 'Precipitação (mm) por Período em ' + mes_guardado + ' na cidade de ' + cidade_guardada)
        src = pil_image_dia
        colors['background'] = '#99EEF1'
        colors['text'] = '#0C3132'
        style_tudo={'backgroundColor': '#99EEF1'}
        style_titulo = {'textAlign': 'center', 'color': '#0C3132', 'fontSize': 45}
        style_texto = {'textAlign': 'center', 'color': '#0C3132', 'fontSize': 30}
        style_texto_atividades = {'textAlign': 'center', 'color': '#0C3132', 'fontSize': 20}
        style_texto_atividades_escolhidas = {'textAlign': 'center', 'color': '#0C3132', 'fontSize': 20}
    else:
        df_essemes = df_grouped[(df_grouped["Mês"] == mes_guardado) & (df_grouped["Periodo"] == período_guardado)]
        figt = px.line(df_essemes, x="Data", y="Temp_Max", color="Periodo", symbol = 'Periodo',title = 'Temperatura Máxima (ºC) por Período em ' + mes_guardado + ' no período de ' + período_guardado + ' na cidade de ' + cidade_guardada)
        figchu = px.line(df_essemes, x="Data", y="Precipitacao", color="Periodo", symbol = 'Periodo',title = 'Precipitação (mm) por Período em ' + mes_guardado + ' no período de ' + período_guardado + ' na cidade de ' + cidade_guardada)
        if período_guardado == 'Noite' or período_guardado == 'Madrugada':
            src = pil_image_noite
            style_tudo={'backgroundColor': '#334B74'}
            style_titulo = {'textAlign': 'center', 'color': '#F0F0F0', 'fontSize': 45}
            style_texto = {'textAlign': 'center', 'color': '#F0F0F0', 'fontSize': 30}
            style_texto_atividades = {'textAlign': 'center', 'color': '#F0F0F0', 'fontSize': 20}
            style_texto_atividades_escolhidas = {'textAlign': 'center', 'color': '#F0F0F0', 'fontSize': 20}
        
        elif período_guardado == 'Tarde':
            src = pil_image_tarde
            style_tudo={'backgroundColor': '#FFA601'}
            style_titulo = {'textAlign': 'center', 'color': '#0C3132', 'fontSize': 45}
            style_texto = {'textAlign': 'center', 'color': '#0C3132', 'fontSize': 30}
            style_texto_atividades = {'textAlign': 'center', 'color': '#0C3132', 'fontSize': 20}
            style_texto_atividades_escolhidas = {'textAlign': 'center', 'color': '#0C3132', 'fontSize': 20}
            
        else:
            src = pil_image_dia
            style_tudo={'backgroundColor': '#99EEF1'}
            style_titulo = {'textAlign': 'center', 'color': '#0C3132', 'fontSize': 45}
            style_texto = {'textAlign': 'center', 'color': '#0C3132', 'fontSize': 30}
            style_texto_atividades = {'textAlign': 'center', 'color': '#0C3132', 'fontSize': 20}
            style_texto_atividades_escolhidas = {'textAlign': 'center', 'color': '#0C3132', 'fontSize': 20}

    return figt, figchu, src, style_tudo, style_titulo, style_texto, style_texto_atividades, style_texto_atividades_escolhidas, texto_atividades



if __name__ == '__main__':
    app.run_server(debug=True, port = 8050)
